In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


def generate_answer(question, answer_paths):

    prompt = f"Question: {question}?\n"
    prompt += f"Paths: {answer_paths}\n"
    prompt += "Answer to question: "

    messages = [
        {"role": "system", "content": "You will be given a question and a list of paths drawn from a knowledge graph. Each of these paths leads from a question entity to an answer to the question. Please use these paths to give all correct answers to the question in a list format."},
        {"role": "user", "content": prompt},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=512,
    )
    print(f"Question: {question}")
    answer = outputs[0]["generated_text"][-1]['content']
    print(f"Answer to question: {answer}")


answer_paths = "\n".join([" -> ".join(path) for path in answer_path_tuples])
question = data[id]['question']
generate_answer(question, answer_paths)

In [12]:
import torch
from content.SubgraphRAG.retrive.src.dataset.retriever import collate_retriever
from content.SubgraphRAG.retrive.src.setup import prepare_sample

def get_top_k_predicted_triples(raw_example, model, device, max_K=500):
    """
    Given a raw example and a trained model, returns the top K predicted triples.

    Parameters:
    - raw_example: dict, a single example containing necessary fields.
    - model: torch.nn.Module, the trained Retriever model.
    - device: torch.device, the device to run the model on.
    - max_K: int, the maximum number of top triples to return.

    Returns:
    - top_K_triples: list of tuples (head_entity, relation, tail_entity, score)
    """
    # Prepare the sample
    sample = collate_retriever([raw_example])
    h_id_tensor, r_id_tensor, t_id_tensor, q_emb, entity_embs,\
        num_non_text_entities, relation_embs, topic_entity_one_hot,\
        target_triple_probs, a_entity_id_list = prepare_sample(device, sample)

    entity_list = raw_example['text_entity_list'] + raw_example['non_text_entity_list']
    relation_list = raw_example['relation_list']
    top_K_triples = []

    if len(h_id_tensor) != 0:
        # Run the model to get logits
        pred_triple_logits = model(
            h_id_tensor.to(device),
            r_id_tensor.to(device),
            t_id_tensor.to(device),
            q_emb.to(device),
            entity_embs.to(device),
            num_non_text_entities,
            relation_embs.to(device),
            topic_entity_one_hot.to(device)
        )
        # Apply sigmoid to get scores between 0 and 1
        pred_triple_scores = torch.sigmoid(pred_triple_logits).reshape(-1)
        # Get top K scores and their indices
        top_K_results = torch.topk(pred_triple_scores, min(max_K, len(pred_triple_scores)))
        top_K_scores = top_K_results.values.cpu().tolist()
        top_K_triple_IDs = top_K_results.indices.cpu().tolist()

        for j, triple_id in enumerate(top_K_triple_IDs):
            top_K_triples.append((
                entity_list[h_id_tensor[triple_id].item()],
                relation_list[r_id_tensor[triple_id].item()],
                entity_list[t_id_tensor[triple_id].item()],
                top_K_scores[j]
            ))

    return top_K_triples


ModuleNotFoundError: No module named 'content'

In [3]:
import torch
import os
print(os.getcwd())

from src.dataset.retriever import RetrieverDataset, collate_retriever
from src.model.retriever import Retriever

def load_model(checkpoint_path, device):
    """
    Loads the trained Retriever model from a checkpoint.

    Parameters:
    - checkpoint_path: str, path to the saved model checkpoint.
    - device: torch.device, the device to load the model on.

    Returns:
    - model: torch.nn.Module, the loaded Retriever model.
    - config: dict, the configuration used during training.
    """
    cpt = torch.load(checkpoint_path, map_location=device)
    config = cpt['config']
    set_seed(config['env']['seed'])
    torch.set_num_threads(config['env']['num_threads'])

    # You may need to adjust this part based on how 'q_emb' is stored
    emb_size = cpt.get('emb_size', 768)  # Default embedding size if not specified

    model = Retriever(emb_size, **config['retriever']).to(device)
    model.load_state_dict(cpt['model_state_dict'])
    model.eval()

    return model, config


/home/gridsan/mhadjiivanov/meng/SubgraphRAG/retrieve


/home/gridsan/mhadjiivanov/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
from src.dataset.retriever import collate_retriever
from src.setup import prepare_sample

def get_top_k_predicted_triples(raw_example, model, device, max_K=500):
    """
    Given a raw example and a trained model, returns the top K predicted triples.

    Parameters:
    - raw_example: dict, a single example containing necessary fields.
    - model: torch.nn.Module, the trained Retriever model.
    - device: torch.device, the device to run the model on.
    - max_K: int, the maximum number of top triples to return.

    Returns:
    - top_K_triples: list of tuples (head_entity, relation, tail_entity, score)
    """
    # Prepare the sample
    sample = collate_retriever([raw_example])
    h_id_tensor, r_id_tensor, t_id_tensor, q_emb, entity_embs,\
        num_non_text_entities, relation_embs, topic_entity_one_hot,\
        target_triple_probs, a_entity_id_list = prepare_sample(device, sample)

    entity_list = raw_example['text_entity_list'] + raw_example['non_text_entity_list']
    relation_list = raw_example['relation_list']
    top_K_triples = []

    if len(h_id_tensor) != 0:
        # Run the model to get logits
        pred_triple_logits = model(
            h_id_tensor.to(device),
            r_id_tensor.to(device),
            t_id_tensor.to(device),
            q_emb.to(device),
            entity_embs.to(device),
            num_non_text_entities,
            relation_embs.to(device),
            topic_entity_one_hot.to(device)
        )
        # Apply sigmoid to get scores between 0 and 1
        pred_triple_scores = torch.sigmoid(pred_triple_logits).reshape(-1)
        # Get top K scores and their indices
        top_K_results = torch.topk(pred_triple_scores, min(max_K, len(pred_triple_scores)))
        top_K_scores = top_K_results.values.cpu().tolist()
        top_K_triple_IDs = top_K_results.indices.cpu().tolist()

        for j, triple_id in enumerate(top_K_triple_IDs):
            top_K_triples.append((
                entity_list[h_id_tensor[triple_id].item()],
                relation_list[r_id_tensor[triple_id].item()],
                entity_list[t_id_tensor[triple_id].item()],
                top_K_scores[j]
            ))

    return top_K_triples


In [5]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Path to your saved model checkpoint
checkpoint_path = "metaqa_Nov20-15:00:11/cpt.pth"


cpt = torch.load(checkpoint_path, map_location='cpu')
config = cpt['config']

infer_set = RetrieverDataset(
        config=config, split='test', skip_no_path=False)

emb_size = infer_set[0]['q_emb'].shape[-1]
model = Retriever(emb_size, **config['retriever']).to(device)
model.load_state_dict(cpt['model_state_dict'])
model = model.to(device)
model.eval()

100%|██████████| 14274/14274 [00:02<00:00, 6784.05it/s]


# skipped samples: 0
# relevant triples | median: 0 | mean: 0 | max: 6


Retriever(
  (non_text_entity_emb): Embedding(1, 1024)
  (dde): DDE(
    (layers): ModuleList(
      (0-1): 2 x PEConv()
    )
    (reverse_layers): ModuleList(
      (0-1): 2 x PEConv()
    )
  )
  (pred): Sequential(
    (0): Linear(in_features=4116, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [47]:
sys_prompt = (
    "Based on the triplets from a knowledge graph, please answer the given question. "
    "Please keep the answers as simple as possible and return all the possible answers "
    """as a list, each with a prefix "ans:"."""
)
cot_prompt = (
    'Format your above answers by listing each answer on a separate line, starting with the prefix "ans:".'
)

sys_prompt_noRAG = (
    "Please answer the given question. "
    "Please keep the answers as simple as possible and return all the possible answers "
    """as a list, each with a prefix "ans:"."""
)

sys_prompt_triplet = (
    "Based on the triplets from a knowledge graph, please answer the given question. "
    "For each answer, provide 3 triples that defend your answer's plausibility."
    "Please keep the answers as simple as possible and return all the possible answers "
    """as a list, each with a prefix "ans: followed by triples:"."""
)

cot_prompt_triplet = (
    'From the answer above, filter out the answers to the question by listing each answer on a separate line, starting with the prefix "ans:".'
)


def triplet_to_str(triplet):
    return f"({triplet[0]},{triplet[1]},{triplet[2]})"

def get_outputs(outputs, model_name):
    return outputs[0]['generated_text'][-1]['content']

    assistant_response = next(
    (message['content'] for message in outputs if message['role'] == 'assistant'),
    None
    )
    return assistant_response
    if "gpt" not in model_name:
        return outputs[0].outputs[0].text
    else:
        return outputs.choices[0].message.content

def get_response(model, raw_example, input_triplets):
    question_prompt = "Question:\n" + raw_example['question']
    if question_prompt[-1] != '?':
        question_prompt += '?'
        
    input_triplets = [triplet_to_str(triplet) for triplet in input_triplets]
    triplet_prompt = "Triplets:\n" + "\n".join(input_triplets)
    
    user_query = "\n\n".join([triplet_prompt, question_prompt])
    print(user_query)
    
    res = []
    
    conversation = [{"role": "system", "content": sys_prompt}]
    conversation.append({"role": "user", "content": user_query})

    outputs = get_outputs(llm(conversation, max_new_tokens=4000, temperature = 0.5), "")
    res.append(outputs)
    
    '''print('pre-CoT Answer')
    print(question_prompt)
    print()
    print(res[-1])
    print()
    print(f"Actual answer: {raw_example['a_entity']}")
    print()'''
    
    conversation.append({"role": "assistant", "content": outputs})
    conversation.append({"role": "user", "content": cot_prompt})
    outputs = get_outputs(llm(conversation, max_new_tokens=4000), model_name)
    res.append(outputs)
    
    print('SubgraphRAG Answer')
    print(question_prompt)
    print()
    print(res[-1])
    print()
    print(f"Actual answer: {raw_example['a_entity']}")
    
def get_response_noRAG(model, raw_example):
    question_prompt = "Question:\n" + raw_example['question']
    if question_prompt[-1] != '?':
        question_prompt += '?'
    
    user_query = "\n\n".join([question_prompt])
    
    res = []
    
    conversation = [{"role": "system", "content": sys_prompt_noRAG}]
    conversation.append({"role": "user", "content": user_query})

    outputs = get_outputs(llm(conversation, max_new_tokens=4000, temperature = 0.5), "")
    res.append(outputs)
    
    conversation.append({"role": "assistant", "content": outputs})
    conversation.append({"role": "user", "content": cot_prompt})
    outputs = get_outputs(llm(conversation, max_new_tokens=4000), model_name)
    print()
    print('No RAG Answer')
    print(question_prompt)
    print()
    print(res[-1])
    print()
    print(f"Actual answer: {raw_example['a_entity']}")
    print()
    
def get_response_triple(model, raw_example, input_triplets):
    question_prompt = "Question:\n" + raw_example['question']
    if question_prompt[-1] != '?':
        question_prompt += '?'
        
    input_triplets = [triplet_to_str(triplet) for triplet in input_triplets]
    triplet_prompt = "Triplets:\n" + "\n".join(input_triplets)
    
    user_query = "\n\n".join([triplet_prompt, question_prompt])
    
    res = []
    
    conversation = [{"role": "system", "content": sys_prompt_triplet}]
    conversation.append({"role": "user", "content": user_query})

    outputs = get_outputs(llm(conversation, max_new_tokens=4000, temperature = 0.5), "")
    res.append(outputs)
    
    '''print('Subgraph Backing preCoT')
    print(question_prompt)
    print()
    print(res[-1])
    print()
    print(f"Actual answer: {raw_example['a_entity']}")
    print()'''
    
    conversation.append({"role": "assistant", "content": outputs})
    conversation.append({"role": "user", "content": cot_prompt_triplet})
    outputs = get_outputs(llm(conversation, max_new_tokens=4000), model_name)
    res.append(outputs)
    
    print()
    print('SubgraphRAG with Triple backing')
    print(question_prompt)
    print()
    print(res[-1])
    print()
    print(f"Actual answer: {raw_example['a_entity']}")

In [7]:
from transformers import pipeline

model_name = '/home/gridsan/mhadjiivanov/meng/SubgraphRAG/hf/models/Llama-3.2-3B-Instruct'
llm = pipeline("text-generation", model=model_name,device=device)


Loading checkpoint shards: 100%|██████████| 2/2 [01:03<00:00, 31.87s/it]


In [44]:
def SubgraphRAG_demo(raw_example, K):
    top_K_triples = get_top_k_predicted_triples(raw_example, model, device, max_K=K)
    answer_coverage = 0
    for answer in raw_example['a_entity']:
        for h,r,t,_ in top_K_triples:
            if t == answer:
                answer_coverage += 1
                break
    answer_coverage = answer_coverage/len(raw_example['a_entity'])
    print(f"% of answers covered: {answer_coverage}")
    print()
    get_response(llm, raw_example, top_K_triples)
    #get_response_triple(llm, raw_example, top_K_triples)
    get_response_noRAG(llm,raw_example)

In [49]:
raw_example = infer_set[15]
SubgraphRAG_demo(raw_example,500)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


% of answers covered: 1.0

Triplets:
(Trog,has_genre,Horror)
(War,has_genre,Action)
(Trog,written_by,John Gilling)
(1941,has_genre,Comedy)
(Trog,directed_by,Freddie Francis)
(Trog,release_year,1970)
(Trog,written_by,Aben Kandel)
(The Challenge,has_genre,Action)
(House,has_genre,Horror)
(Asylum,has_genre,Horror)
(I Was a Male War Bride,has_genre,War)
(Trog,starred_actors,Joan Crawford)
(Das Millionenspiel,has_genre,Action)
(Planet Terror,has_genre,Action)
(V/H/S,has_genre,Horror)
(It's Alive,has_genre,Horror)
(Black Death,has_genre,Action)
(Macabre,has_genre,Thriller)
(Planet Terror,has_genre,Horror)
(MASH,has_genre,War)
(The Alligator People,has_genre,Horror)
(Catch-22,has_genre,War)
(Tormented,has_genre,Horror)
(Humoresque,has_genre,Drama)
(Asylum,has_genre,Drama)
(Performance,has_genre,Crime)
(Them!,has_genre,Horror)
(Black Death,has_genre,Horror)
(House,has_genre,Comedy)
(Invaders from Mars,has_genre,Horror)
(Cat People,has_genre,Horror)
(Halloween,has_genre,Horror)
(Macabre,has_gen

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


SubgraphRAG Answer
Question:
what genres are the movies starred by Trog actors?

ans: Drama
ans: Horror
ans: Thriller

Actual answer: ['Musical', 'Western', 'Comedy', 'Thriller', 'Horror', 'Music', 'Fantasy', 'Drama']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



No RAG Answer
Question:
what genres are the movies starred by Trog actors?

I couldn't find any information on "Trog actors." Could you please provide more context or clarify who Trog actors are? I'll do my best to provide a list of genres for movies starred by that group.

Actual answer: ['Musical', 'Western', 'Comedy', 'Thriller', 'Horror', 'Music', 'Fantasy', 'Drama']

